In [576]:
from bs4 import BeautifulSoup
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [577]:
URL = "https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310539/bs.html#ASFSK"
    

In [578]:
def team_checker(driver):
    home_team = driver.find_elements(By.TAG_NAME, "tr")
    home_team_finder = home_team[0].text
    
    if home_team_finder.split(" ")[0] == "ITS":
        return True
    return False

In [579]:
def soup_initializer(URL): 
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, "html")
    return soup

In [580]:
def initialize_driver(URL):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    return driver

In [581]:
def flatten_and_append(data, new_list=None):
  if new_list is None:
    new_list = []
  
  for item in data:
    if isinstance(item, list):
      new_list = flatten_and_append(item, new_list)
    else:
      new_list.append(item)

  return new_list

In [582]:
def selenium_extractor_bench(driver):
    element_bench = driver.find_elements(By.CLASS_NAME, "bench")
    player_stats_bench = element_bench[0].text
    
    return player_stats_bench

In [583]:
def selenium_extractor_starters(driver):
    element_starters = driver.find_elements(By.TAG_NAME, "tbody")
    player_stats_starters = element_starters[1].text
    
    return player_stats_starters

In [584]:
def dictionary_creator(boxscore_section):

  player_stats_splitted = boxscore_section.split("\n")

  for i in range(0, len(player_stats_splitted), 3):
    player_stats_splitted[i+2] = player_stats_splitted[i+2].split(" ")
      
  flattened_list = flatten_and_append(player_stats_splitted)
  starters = {f"starter_{i + 1}": flattened_list[i:i+24] for i in range(0, len(flattened_list), 24)}

  return starters

In [585]:
def dataframe_inserter(dictionary, dataframe):
    for key, value in dictionary.items():
        dataframe.loc[len(dataframe)] = value
    return dataframe

In [586]:
def column_extractor(soup):    
    boxscore = soup.find_all("table")[1]
    column_names = []
    headers = boxscore.find("thead").findAll("th")

    for th in headers:
        column_names.append(th.text.strip())
    column_names = column_names[:24]
    
    return column_names

In [587]:
def main():
    
    from bs4 import BeautifulSoup
    import requests
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.by import By
    from selenium.webdriver import ChromeOptions
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    import pandas as pd
    
    URL = "https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310539/bs.html#ASFSK"
    
    soup = soup_initializer(URL)
    driver = initialize_driver(URL)
    
    column_names = column_extractor(soup)
    df = pd.DataFrame(columns= column_names)
    
    if team_checker(driver) == True:
    
        try:
            starter_stats = selenium_extractor_starters(driver)
            player_stats_starters = dictionary_creator(starter_stats)
            
            
            bench_stats = selenium_extractor_bench(driver)
            player_stats_bench = dictionary_creator(bench_stats)
            
            df = dataframe_inserter(player_stats_starters, df)
            df = dataframe_inserter(player_stats_bench, df)
            
        finally:
            driver.quit()
        
    return df

In [588]:
main()

,No.,Jugador,POS,MIN,Pt,TC,Ti%,2P,2P%,3P,...,RD,RT,AS,PER,REC,Tap,TapRe,FP,FR,+-Pt
0,1,J. Sanchez,G,21:25,0,0-6,0,0-3,0,0-3,...,4,5,3,1,3,0,0,1,0,15
1,11,J. Sanchez,G,18:20,4,2-8,25,2-5,40,0-3,...,4,5,2,3,0,0,0,3,1,16
2,13,R. Flores,G,27:06,21,8-13,61,7-10,70,1-3,...,4,5,2,2,2,0,0,0,5,16
3,26,A. Gallardo,F,20:18,11,5-11,45,5-11,45,0-0,...,4,5,1,1,1,0,0,2,1,20
4,37,R. De La Mora,F,23:57,9,4-8,50,3-6,50,1-2,...,4,6,2,1,0,0,0,4,0,17
5,3,M. Cardenas,G,7:30,2,1-4,25,1-3,33,0-1,...,1,3,0,0,1,0,0,3,3,4
6,9,C. Torres,G,16:12,5,2-9,22,1-3,33,1-6,...,0,0,0,1,1,0,0,2,2,6
7,10,J. Puente,PG,14:52,4,2-3,66,2-2,100,0-1,...,4,5,0,2,3,0,0,2,0,9
8,12,J. Michel,C,18:07,4,2-6,33,2-6,33,0-0,...,5,7,2,1,2,0,0,3,0,20
9,20,C. Riolo,G,10:08,0,0-2,0,0-2,0,0-0,...,0,2,0,0,0,0,0,2,1,4
